In [91]:
from Bio import Restriction
from Bio.Seq import Seq
from Bio import SeqIO
import contextlib
import io
from statistics import mean
from statistics import median
import json


In [8]:
test_seq = {}
with open("./random_dna_1.fa") as dna:
    for record in SeqIO.parse(dna, "fasta"):
        test_seq[record.id] = record.seq
print(test_seq)

{'test1': Seq('TCATTAACTGTTGTTTGGTAGCACAAAAGTATTACCATGGTCCTAGAAGTTCGG...ATT'), 'test2': Seq('CGTTATGGCACCAGGGAGTTTAAGCCGAGTCAATGGAGCTCGCAATACAGAGTT...GCC')}


In [98]:
enzymes = {str(x): getattr(Restriction, str(x)) for x in sorted(list(Restriction.CommOnly))}
# print(enzymes)
enzyme_obj = {}

for enzyme in enzymes:
  # if "A" in enzymes[enzyme].site:
    # print(enzyme, enzymes[enzyme].site, enzymes[enzyme].overhang(), enzymes[enzyme].cut_once(), enzymes[enzyme].cut_twice(), enzymes[enzyme].inact_temp , enzymes[enzyme].is_methylable())#, [x for x in dir(enzymes[enzyme]) if not x.startswith("_")])
    enzyme_obj[str(enzyme)] = {
      "site": enzymes[enzyme].site,
      "overhang": enzymes[enzyme].overhang(),
      "cut_once": enzymes[enzyme].cut_once(),
      "cut_twice": enzymes[enzyme].cut_twice(),
      "inact_temp": enzymes[enzyme].inact_temp,
      "is_methylable": enzymes[enzyme].is_methylable(),
    } 
print(enzyme_obj)

# jsonify the enzyme object
with open("enzyme.json", "w") as f:
  json.dump(enzyme_obj, f, indent=2)
  
# Check the values of each object in the enzyme object
site_vals = set()
overhang_vals = set()
cut_once_vals = set()
cut_twice_vals = set()
inact_temp_vals = set()
is_methylable_vals = set()

for enzyme in enzyme_obj:
  site_vals.add(enzyme_obj[enzyme]["site"])
  overhang_vals.add(enzyme_obj[enzyme]["overhang"])
  cut_once_vals.add(enzyme_obj[enzyme]["cut_once"])
  cut_twice_vals.add(enzyme_obj[enzyme]["cut_twice"])
  inact_temp_vals.add(enzyme_obj[enzyme]["inact_temp"])
  is_methylable_vals.add(enzyme_obj[enzyme]["is_methylable"])

print(f"site_vals: {site_vals}, lenght: {len(site_vals)}")
print(f"overhang_vals: {overhang_vals}, lenght: {len(overhang_vals)}")
print(f"cut_once_vals: {cut_once_vals}, lenght: {len(cut_once_vals)}")
print(f"cut_twice_vals: {cut_twice_vals}, lenght: {len(cut_twice_vals)}")
print(f"inact_temp_vals: {inact_temp_vals}, lenght: {len(inact_temp_vals)}")
print(f"is_methylable_vals: {is_methylable_vals}, lenght: {len(is_methylable_vals)}")



{'AbaSI': {'site': 'C', 'overhang': "3' overhang", 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': False}, 'FspEI': {'site': 'CC', 'overhang': "5' overhang", 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': False}, 'AccII': {'site': 'CGCG', 'overhang': 'blunt', 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': False}, 'AciI': {'site': 'CCGC', 'overhang': "5' overhang", 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': True}, 'AfaI': {'site': 'GTAC', 'overhang': 'blunt', 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': False}, 'AluBI': {'site': 'AGCT', 'overhang': 'blunt', 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': True}, 'AluI': {'site': 'AGCT', 'overhang': 'blunt', 'cut_once': True, 'cut_twice': False, 'inact_temp': 65, 'is_methylable': True}, 'AoxI': {'site': 'GGCC', 'overhang': "5' overhang", 'cut_once': True, 'cut_twice': False, 'inact

In [13]:
test_analysis = Restriction.Analysis(Restriction.CommOnly, test_seq["test1"], linear=True)

In [19]:
test_analysis.print_as("map")
test_analysis.print_that()

 2 MspJI
 |                                                          
 | 4 AbaSI
 | |                                                        
 | |5 Tru9I MseI SaqAI Tru1I
 | ||                                                       
 | ||  8 AbaSI
 | ||  |                                                    
 | ||  |9 AbaSI
 | ||  ||                                                   
 | ||  ||10 TaaI Bst4CI HpyCH4III
 | ||  |||                                                  
 | ||  ||| 12 AbaSI
 | ||  ||| |                                                
 | ||  ||| | 14 AbaSI
 | ||  ||| | |                                              
 | ||  ||| | |     20 AbaSI AbaSI
 | ||  ||| | |     |                                        
 | ||  ||| | |     |  23 FspEI
 | ||  ||| | |     |  |                                     
 | ||  ||| | |     |  |      30 AbaSI
 | ||  ||| | |     |  |      |                              
 | ||  ||| | |     |  |      |31 AbaSI
 | ||  ||| | |     |  |      ||

In [26]:
test_analysis.print_as("map")
test_str = str(test_analysis.print_that())
# print(test_str)

 2 MspJI
 |                                                          
 | 4 AbaSI
 | |                                                        
 | |5 Tru9I MseI SaqAI Tru1I
 | ||                                                       
 | ||  8 AbaSI
 | ||  |                                                    
 | ||  |9 AbaSI
 | ||  ||                                                   
 | ||  ||10 TaaI Bst4CI HpyCH4III
 | ||  |||                                                  
 | ||  ||| 12 AbaSI
 | ||  ||| |                                                
 | ||  ||| | 14 AbaSI
 | ||  ||| | |                                              
 | ||  ||| | |     20 AbaSI AbaSI
 | ||  ||| | |     |                                        
 | ||  ||| | |     |  23 FspEI
 | ||  ||| | |     |  |                                     
 | ||  ||| | |     |  |      30 AbaSI
 | ||  ||| | |     |  |      |                              
 | ||  ||| | |     |  |      |31 AbaSI
 | ||  ||| | |     |  |      ||

In [24]:
print(test_str)

None
